In [1]:
import FiOps_Functions as Funcs
import pandas as pd
import numpy as np
import time
import datetime
from datetime import datetime
import sqlalchemy

In [2]:
conn = Funcs.sql_fiops_connect()

Please enter your Sungevity username: nseelam


Please enter your Database Password: 
Note: Please copy and paste your password, by right click pasting. Ctrl - V does not work in the Command Prompt
········


In [4]:
Updated_Date = """ select Last_Refreshed_Date from fiops.milestone_velocity_reverse_weekly LIMIT 1"""
Last_Refreshed_Date = pd.read_sql_query(Updated_Date, conn)
Last_Refreshed_Date = Last_Refreshed_Date.loc[0, 'Last_Refreshed_Date']
Last_Refreshed_Date

datetime.date(2017, 2, 6)

In [5]:
today = datetime.today()
today = today.strftime("%Y-%m-%d")
today = datetime.strptime(today,"%Y-%m-%d" ).date()
#date = datetime.datetime.strptime('2012-02-11' , '%Y-%m-%d')
today

datetime.date(2017, 2, 6)

In [6]:
kinaM1query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	M.SALE_CLOSED__C as 'Sale_Closed_Date'
,	FPA.M1_Submitted_Date as 'M1_Submitted_Date'
,	YEAR(FPA.M1_Submitted_Date) as 'Year'
,	MONTH(FPA.M1_Submitted_Date) as 'Month'
,	DAY(FPA.M1_Submitted_Date) as 'Day'
,	DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) as 'Days_to_M1'
,	'Kinaole' as 'Fund'
,	'Sale - M1' as 'Milestone'
,	CASE
	WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -30 THEN 0
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -60 THEN 1
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -90 THEN 2
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -120 THEN 3
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -150 THEN 4
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -180 THEN 5
    ELSE 6 
    END as 'Months_to_M1'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON (t.id = M.Tranche__c)
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON (FP.Project__c = M.Id AND FP.Selected_Product__c = 1)
        -- LEFT JOIN Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute FPA ON FP.Id = FPA.Funding_Product__c
        
WHERE 
FP.name IN ('kinaole_lease','kinaole_ppa','sunfinco_lease','sunfinco_ppa')
AND M.IS_TEST__C = 0
AND t.Name LIKE 'Kinaole%%'
AND M.STATUS__C <> 'Cancelled'
AND FPA.M1_Submitted_Date IS NOT NULL
"""

kinaM2query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	FPA.M1_Submitted_Date as 'M1_Submitted_Date'
,	FPA.M2_Submitted_Date as 'M2_Submitted_Date'
,	YEAR(FPA.M2_Submitted_Date) as 'Year'
,	MONTH(FPA.M2_Submitted_Date) as 'Month'
,	DAY(FPA.M2_Submitted_Date) as 'Day'
,	DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) as 'Days_M1_M2'
,	'Kinaole' as 'Fund'
,	'M1 - M2' as 'Milestone'
,	CASE
	WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -30 THEN 0
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -60 THEN 1
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -90 THEN 2
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -120 THEN 3
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -150 THEN 4
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -180 THEN 5
    ELSE 6 
    END as 'Months_M1_M2'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON (t.id = M.Tranche__c)
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON (FP.Project__c = M.Id AND FP.Selected_Product__c = 1)
        -- LEFT JOIN Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute FPA ON FP.Id = FPA.Funding_Product__c
        
WHERE 
FP.name IN ('kinaole_lease','kinaole_ppa','sunfinco_lease','sunfinco_ppa')
AND M.IS_TEST__C = 0
AND t.Name LIKE 'Kinaole%%'
AND M.STATUS__C <> 'Cancelled'
AND FPA.M1_Submitted_Date IS NOT NULL
AND FPA.M2_Submitted_Date IS NOT NULL
"""

kinaM3query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	FPA.M2_Submitted_Date as 'M2_Submitted_Date'
,	FPA.M3_Submitted_Date as 'M3_Submitted_Date'
,	YEAR(FPA.M3_Submitted_Date) as 'Year'
,	MONTH(FPA.M3_Submitted_Date) as 'Month'
,	DAY(FPA.M3_Submitted_Date) as 'Day'
,	DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) as 'Days_M2_M3'
,	'Kinaole' as 'Fund'
,	'M2 - M3' as 'Milestone'
,	CASE
	WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -30 THEN 0
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -60 THEN 1
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -90 THEN 2
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -120 THEN 3
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -150 THEN 4
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -180 THEN 5
    ELSE 6 
    END as 'Months_M2_M3'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON (t.id = M.Tranche__c)
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON (FP.Project__c = M.Id AND FP.Selected_Product__c = 1)
        -- LEFT JOIN Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute FPA ON FP.Id = FPA.Funding_Product__c
        
WHERE 
FP.name IN ('kinaole_lease','kinaole_ppa','sunfinco_lease','sunfinco_ppa')
AND M.IS_TEST__C = 0
AND t.Name LIKE 'Kinaole%%'
AND M.STATUS__C <> 'Cancelled'
AND FPA.M2_Submitted_Date IS NOT NULL
AND FPA.M3_Submitted_Date IS NOT NULL
"""

In [7]:
sr2M1query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	M.SALE_CLOSED__C as 'Sale_Closed_Date'
,	FPA.M1_Submitted_Date as 'M1_Submitted_Date'
,	YEAR(FPA.M1_Submitted_Date) as 'Year'
,	MONTH(FPA.M1_Submitted_Date) as 'Month'
,	DAY(FPA.M1_Submitted_Date) as 'Day'
,	DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) as 'Days_to_M1'
,	'Sunrun 2' as 'Fund'
,	'Sale - M1' as 'Milestone'
,	CASE
	WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -30 THEN 0
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -60 THEN 1
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -90 THEN 2
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -120 THEN 3
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -150 THEN 4
    WHEN DATEDIFF(M.SALE_CLOSED__C, FPA.M1_Submitted_Date) > -180 THEN 5
    ELSE 6 
    END as 'Months_to_M1'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON (t.id = M.Tranche__c)
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON (FP.Project__c = M.Id AND FP.Selected_Product__c = 1)
        -- LEFT JOIN Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute FPA ON FP.Id = FPA.Funding_Product__c
        
WHERE 
M.TRANCHE_ID__C IN ('P1','P2','P3','P4','P5','P6','P7','P8','P9','P10','P11','P12','P13','P14','P15','P16','P17','P18','P19','P20')
AND M.IS_TEST__C = 0
AND M.STATUS__C <> 'Cancelled'
AND FPA.M1_Submitted_Date IS NOT NULL
"""

sr2M2query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	FPA.M1_Submitted_Date as 'M1_Submitted_Date'
,	FPA.M2_Submitted_Date as 'M2_Submitted_Date'
,	YEAR(FPA.M2_Submitted_Date) as 'Year'
,	MONTH(FPA.M2_Submitted_Date) as 'Month'
,	DAY(FPA.M2_Submitted_Date) as 'Day'
,	DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) as 'Days_M1_M2'
,	'Sunrun 2' as 'Fund'
,	'M1 - M2' as 'Milestone'
,	CASE
	WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -30 THEN 0
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -60 THEN 1
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -90 THEN 2
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -120 THEN 3
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -150 THEN 4
    WHEN DATEDIFF(FPA.M1_Submitted_Date, FPA.M2_Submitted_Date) > -180 THEN 5
    ELSE 6 
    END as 'Months_M1_M2'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON (t.id = M.Tranche__c)
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON (FP.Project__c = M.Id AND FP.Selected_Product__c = 1)
        -- LEFT JOIN Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute FPA ON FP.Id = FPA.Funding_Product__c
        
WHERE 
M.TRANCHE_ID__C IN ('P1','P2','P3','P4','P5','P6','P7','P8','P9','P10','P11','P12','P13','P14','P15','P16','P17','P18','P19','P20')
AND M.IS_TEST__C = 0
AND M.STATUS__C <> 'Cancelled'
AND FPA.M1_Submitted_Date IS NOT NULL
AND FPA.M2_Submitted_Date IS NOT NULL
"""

sr2M3query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	FPA.M2_Submitted_Date as 'M2_Submitted_Date'
,	FPA.M3_Submitted_Date as 'M3_Submitted_Date'
,	YEAR(FPA.M3_Submitted_Date) as 'Year'
,	MONTH(FPA.M3_Submitted_Date) as 'Month'
,	DAY(FPA.M3_Submitted_Date) as 'Day'
,	DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) as 'Days_M2_M3'
,	'Sunrun 2' as 'Fund'
,	'M2 - M3' as 'Milestone'
,	CASE
	WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -30 THEN 0
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -60 THEN 1
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -90 THEN 2
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -120 THEN 3
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -150 THEN 4
    WHEN DATEDIFF(FPA.M2_Submitted_Date, FPA.M3_Submitted_Date) > -180 THEN 5
    ELSE 6 
    END as 'Months_M2_M3'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON (t.id = M.Tranche__c)
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON (FP.Project__c = M.Id AND FP.Selected_Product__c = 1)
        -- LEFT JOIN Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute FPA ON FP.Id = FPA.Funding_Product__c
        
WHERE 
M.TRANCHE_ID__C IN ('P1','P2','P3','P4','P5','P6','P7','P8','P9','P10','P11','P12','P13','P14','P15','P16','P17','P18','P19','P20')
AND M.IS_TEST__C = 0
AND M.STATUS__C <> 'Cancelled'
AND FPA.M2_Submitted_Date IS NOT NULL
AND FPA.M3_Submitted_Date IS NOT NULL
"""

In [8]:
mosaicM1query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	M.SALE_CLOSED__C as 'Sale_Closed_Date'
,	YEAR(JRSLPA.F1_Submitted_Date) as 'Year'
,	MONTH(JRSLPA.F1_Submitted_Date) as 'Month'
,	DAY(JRSLPA.F1_Submitted_Date) as 'Day'
,	JRSLPA.F1_Submitted_Date as 'M1_Submitted_Date'
,	DATEDIFF(M.SALE_CLOSED__C, JRSLPA.F1_Submitted_Date) as 'Days_to_M1'
,	'Mosaic' as 'Fund'
,	'Sale - M1' as 'Milestone'
,	CASE
	WHEN DATEDIFF(M.SALE_CLOSED__C, JRSLPA.F1_Submitted_Date) > -30 THEN 0
    WHEN DATEDIFF(M.SALE_CLOSED__C, JRSLPA.F1_Submitted_Date) > -60 THEN 1
    WHEN DATEDIFF(M.SALE_CLOSED__C, JRSLPA.F1_Submitted_Date) > -90 THEN 2
    WHEN DATEDIFF(M.SALE_CLOSED__C, JRSLPA.F1_Submitted_Date) > -120 THEN 3
    WHEN DATEDIFF(M.SALE_CLOSED__C, JRSLPA.F1_Submitted_Date) > -150 THEN 4
    WHEN DATEDIFF(M.SALE_CLOSED__C, JRSLPA.F1_Submitted_Date) > -180 THEN 5
    ELSE 6 
    END as 'Months_to_M1'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
		LEFT JOIN salesforce_prod.Account a ON iQuote__c.Account__c = a.Id
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON M.Tranche__c = t.Id
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON FP.Project__c = M.Id AND FP.Selected_Product__c = 1
        LEFT JOIN salesforce_prod.Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute JRSFPA ON FP.Id = JRSFPA.Funding_Product__c
		LEFT JOIN salesforce_prod.loan_product__c LProd ON iQuote__c.ID = LProd.IQUOTE__C AND LProd.PRIMARY_PRODUCT__C = 1 AND LProd.ACTIVE__C = 1
		LEFT JOIN salesforce_prod.jrs_loan_product_attribute JRSLPA ON LProd.ID = JRSLPA.Loan_Product_Id 
		LEFT JOIN salesforce_prod.loan_provider__c LProv ON LProd.LOAN_PROVIDER__C = LProv.ID 
        
WHERE 
LProv.Name = 'Mosaic'
AND M.IS_TEST__C = 0
AND M.STATUS__C <> 'Cancelled'
AND JRSLPA.F1_Submitted_Date IS NOT NULL
"""

mosaicM2query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	YEAR(JRSLPA.F2_Submitted_Date) as 'Year'
,	MONTH(JRSLPA.F2_Submitted_Date) as 'Month'
,	DAY(JRSLPA.F2_Submitted_Date) as 'Day'
,	JRSLPA.F1_Submitted_Date as 'M1_Submitted_Date'
,	JRSLPA.F2_Submitted_Date as 'M2_Submitted_Date'
,	DATEDIFF(JRSLPA.F1_Submitted_Date,JRSLPA.F2_Submitted_Date) as 'Days_M1_M2'
,	'Mosaic' as 'Fund'
,	'M1 - M2' as 'Milestone'
,	CASE
	WHEN DATEDIFF(JRSLPA.F1_Submitted_Date,JRSLPA.F2_Submitted_Date) > -30 THEN 0
    WHEN DATEDIFF(JRSLPA.F1_Submitted_Date,JRSLPA.F2_Submitted_Date) > -60 THEN 1
    WHEN DATEDIFF(JRSLPA.F1_Submitted_Date,JRSLPA.F2_Submitted_Date) > -90 THEN 2
    WHEN DATEDIFF(JRSLPA.F1_Submitted_Date,JRSLPA.F2_Submitted_Date) > -120 THEN 3
    WHEN DATEDIFF(JRSLPA.F1_Submitted_Date,JRSLPA.F2_Submitted_Date) > -150 THEN 4
    WHEN DATEDIFF(JRSLPA.F1_Submitted_Date,JRSLPA.F2_Submitted_Date) > -180 THEN 5
    ELSE 6 
    END as 'Months_M1_M2'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
		LEFT JOIN salesforce_prod.Account a ON iQuote__c.Account__c = a.Id
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON M.Tranche__c = t.Id
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON FP.Project__c = M.Id AND FP.Selected_Product__c = 1
        LEFT JOIN salesforce_prod.Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute JRSFPA ON FP.Id = JRSFPA.Funding_Product__c
		LEFT JOIN salesforce_prod.loan_product__c LProd ON iQuote__c.ID = LProd.IQUOTE__C AND LProd.PRIMARY_PRODUCT__C = 1 AND LProd.ACTIVE__C = 1
		LEFT JOIN salesforce_prod.jrs_loan_product_attribute JRSLPA ON LProd.ID = JRSLPA.Loan_Product_Id 
		LEFT JOIN salesforce_prod.loan_provider__c LProv ON LProd.LOAN_PROVIDER__C = LProv.ID 
        
WHERE 
LProv.Name = 'Mosaic'
AND M.IS_TEST__C = 0
AND M.STATUS__C <> 'Cancelled'
AND JRSLPA.F1_Submitted_Date IS NOT NULL
AND JRSLPA.F2_Submitted_Date IS NOT NULL
"""

mosaicM3query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	YEAR(JRSLPA.F3_Submitted_Date) as 'Year'
,	MONTH(JRSLPA.F3_Submitted_Date) as 'Month'
,	DAY(JRSLPA.F3_Submitted_Date) as 'Day'
,	JRSLPA.F2_Submitted_Date as 'M2_Submitted_Date'
,	JRSLPA.F3_Submitted_Date as 'M3_Submitted_Date'
,	DATEDIFF(JRSLPA.F2_Submitted_Date,JRSLPA.F3_Submitted_Date) as 'Days_M2_M3'
,	'Mosaic' as 'Fund'
,	'M2 - M3' as 'Milestone'
,	CASE
	WHEN DATEDIFF(JRSLPA.F2_Submitted_Date,JRSLPA.F3_Submitted_Date) > -30 THEN 0
    WHEN DATEDIFF(JRSLPA.F2_Submitted_Date,JRSLPA.F3_Submitted_Date) > -60 THEN 1
    WHEN DATEDIFF(JRSLPA.F2_Submitted_Date,JRSLPA.F3_Submitted_Date) > -90 THEN 2
    WHEN DATEDIFF(JRSLPA.F2_Submitted_Date,JRSLPA.F3_Submitted_Date) > -120 THEN 3
    WHEN DATEDIFF(JRSLPA.F2_Submitted_Date,JRSLPA.F3_Submitted_Date) > -150 THEN 4
    WHEN DATEDIFF(JRSLPA.F2_Submitted_Date,JRSLPA.F3_Submitted_Date) > -180 THEN 5
    ELSE 6 
    END as 'Months_M2_M3'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
		LEFT JOIN salesforce_prod.Account a ON iQuote__c.Account__c = a.Id
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON M.Tranche__c = t.Id
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON FP.Project__c = M.Id AND FP.Selected_Product__c = 1
        LEFT JOIN salesforce_prod.Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute JRSFPA ON FP.Id = JRSFPA.Funding_Product__c
		LEFT JOIN salesforce_prod.loan_product__c LProd ON iQuote__c.ID = LProd.IQUOTE__C AND LProd.PRIMARY_PRODUCT__C = 1 AND LProd.ACTIVE__C = 1
		LEFT JOIN salesforce_prod.jrs_loan_product_attribute JRSLPA ON LProd.ID = JRSLPA.Loan_Product_Id 
		LEFT JOIN salesforce_prod.loan_provider__c LProv ON LProd.LOAN_PROVIDER__C = LProv.ID 
        
WHERE 
LProv.Name = 'Mosaic'
AND M.IS_TEST__C = 0
AND M.STATUS__C <> 'Cancelled'
AND JRSLPA.F2_Submitted_Date IS NOT NULL
AND JRSLPA.F3_Submitted_Date IS NOT NULL
"""

In [9]:
purecashM1query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	M.SALE_CLOSED__C as 'Sale_Closed_Date'
,	YEAR(M.SALE_CLOSED__C) as 'Year'
,	MONTH(M.SALE_CLOSED__C) as 'Month'
,	DAY(M.SALE_CLOSED__C) as 'Day'
,	M.SALE_CLOSED__C as 'M1_Submitted_Date'
,	DATEDIFF(M.SALE_CLOSED__C, M.SALE_CLOSED__C) as 'Days_to_M1'
,	'Pure Cash' as 'Fund'
,	'Sale - M1' as 'Milestone'
,	CASE
	WHEN DATEDIFF(M.SALE_CLOSED__C, M.SALE_CLOSED__C) > -30 THEN 0
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.SALE_CLOSED__C) > -60 THEN 1
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.SALE_CLOSED__C) > -90 THEN 2
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.SALE_CLOSED__C) > -120 THEN 3
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.SALE_CLOSED__C) > -150 THEN 4
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.SALE_CLOSED__C) > -180 THEN 5
    ELSE 6 
    END as 'Months_to_M1'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
		LEFT JOIN salesforce_prod.Account a ON iQuote__c.Account__c = a.Id
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON M.Tranche__c = t.Id
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON FP.Project__c = M.Id AND FP.Selected_Product__c = 1
        LEFT JOIN salesforce_prod.Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute JRSFPA ON FP.Id = JRSFPA.Funding_Product__c
		LEFT JOIN salesforce_prod.loan_product__c LProd ON iQuote__c.ID = LProd.IQUOTE__C AND LProd.PRIMARY_PRODUCT__C = 1 AND LProd.ACTIVE__C = 1
		LEFT JOIN salesforce_prod.jrs_loan_product_attribute JRSLPA ON LProd.ID = JRSLPA.Loan_Product_Id 
		LEFT JOIN salesforce_prod.loan_provider__c LProv ON LProd.LOAN_PROVIDER__C = LProv.ID 
        
WHERE 
LProv.Name IN ('HERO','EFS','Admiral','CAL First')
AND M.IS_TEST__C = 0
AND M.STATUS__C <> 'Cancelled'
AND M.SALE_CLOSED__C IS NOT NULL
"""

purecashM2query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	YEAR( M.PERMIT_APPROVED_DATE__C) as 'Year'
,	MONTH( M.PERMIT_APPROVED_DATE__C) as 'Month'
,	DAY( M.PERMIT_APPROVED_DATE__C) as 'Day'
,	M.SALE_CLOSED__C as 'M1_Submitted_Date'
,	 M.PERMIT_APPROVED_DATE__C as 'M2_Submitted_Date'
,	DATEDIFF(M.SALE_CLOSED__C, M.PERMIT_APPROVED_DATE__C) as 'Days_M1_M2'
,	'Pure Cash' as 'Fund'
,	'M1 - M2' as 'Milestone'
,	CASE
	WHEN DATEDIFF(M.SALE_CLOSED__C, M.PERMIT_APPROVED_DATE__C) > -30 THEN 0
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.PERMIT_APPROVED_DATE__C) > -60 THEN 1
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.PERMIT_APPROVED_DATE__C) > -90 THEN 2
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.PERMIT_APPROVED_DATE__C) > -120 THEN 3
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.PERMIT_APPROVED_DATE__C) > -150 THEN 4
    WHEN DATEDIFF(M.SALE_CLOSED__C, M.PERMIT_APPROVED_DATE__C) > -180 THEN 5
    ELSE 6 
    END as 'Months_M1_M2'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
		LEFT JOIN salesforce_prod.Account a ON iQuote__c.Account__c = a.Id
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON M.Tranche__c = t.Id
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON FP.Project__c = M.Id AND FP.Selected_Product__c = 1
        LEFT JOIN salesforce_prod.Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute JRSFPA ON FP.Id = JRSFPA.Funding_Product__c
		LEFT JOIN salesforce_prod.loan_product__c LProd ON iQuote__c.ID = LProd.IQUOTE__C AND LProd.PRIMARY_PRODUCT__C = 1 AND LProd.ACTIVE__C = 1
		LEFT JOIN salesforce_prod.jrs_loan_product_attribute JRSLPA ON LProd.ID = JRSLPA.Loan_Product_Id 
		LEFT JOIN salesforce_prod.loan_provider__c LProv ON LProd.LOAN_PROVIDER__C = LProv.ID 
        
WHERE 
LProv.Name IN ('HERO','EFS','Admiral','CAL First')
AND M.IS_TEST__C = 0
AND M.STATUS__C <> 'Cancelled'
AND M.PERMIT_APPROVED_DATE__C IS NOT NULL
"""

purecashM3query = """
SELECT
	M.ID as 'ID'
,	M.Name as 'Project Name'
,	YEAR(M.INSTALLATION_COMPLETE_DATE__C) as 'Year'
,	MONTH(M.INSTALLATION_COMPLETE_DATE__C) as 'Month'
,	DAY(M.INSTALLATION_COMPLETE_DATE__C) as 'Day'
,	M.PERMIT_APPROVED_DATE__C as 'M2_Submitted_Date'
,	M.INSTALLATION_COMPLETE_DATE__C as 'M3_Submitted_Date'
,	DATEDIFF(M.PERMIT_APPROVED_DATE__C,M.INSTALLATION_COMPLETE_DATE__C) as 'Days_M2_M3'
,	'Pure Cash' as 'Fund'
,	'M2 - M3' as 'Milestone'
,	CASE
	WHEN DATEDIFF(M.PERMIT_APPROVED_DATE__C,M.INSTALLATION_COMPLETE_DATE__C) > -30 THEN 0
    WHEN DATEDIFF(M.PERMIT_APPROVED_DATE__C,M.INSTALLATION_COMPLETE_DATE__C) > -60 THEN 1
    WHEN DATEDIFF(M.PERMIT_APPROVED_DATE__C,M.INSTALLATION_COMPLETE_DATE__C) > -90 THEN 2
    WHEN DATEDIFF(M.PERMIT_APPROVED_DATE__C,M.INSTALLATION_COMPLETE_DATE__C) > -120 THEN 3
    WHEN DATEDIFF(M.PERMIT_APPROVED_DATE__C,M.INSTALLATION_COMPLETE_DATE__C) > -150 THEN 4
    WHEN DATEDIFF(M.PERMIT_APPROVED_DATE__C,M.INSTALLATION_COMPLETE_DATE__C) > -180 THEN 5
    ELSE 6 
    END as 'Months_M2_M3'

FROM
	salesforce_prod.Opportunity O
		LEFT JOIN salesforce_prod.Lead Lead ON O.ID = lead.CONVERTEDOPPORTUNITYID 
			AND Lead.Name NOT LIKE '%%testsungevity%%'
			AND Lead.Name NOT LIKE '%%sungevitytest%%'
			AND lead.`STATUS` NOT LIKE 'Qualified Duplicate'
		LEFT JOIN salesforce_prod.iQuote__c ON O.Id = iQuote__c.Opportunity__c
		LEFT JOIN salesforce_prod.Account a ON iQuote__c.Account__c = a.Id
        LEFT JOIN salesforce_prod.Milestone1_Project__c M ON M.Id = iQuote__c.Project__c 
        LEFT JOIN salesforce_prod.Tranche__c t ON M.Tranche__c = t.Id
        LEFT JOIN salesforce_prod.Funding_Product__c FP ON FP.Project__c = M.Id AND FP.Selected_Product__c = 1
        LEFT JOIN salesforce_prod.Funding_Product_Attribute_column_view FPA ON FPA.Funding_Product__c = FP.Id
        LEFT JOIN salesforce_prod.Funding_Provider__c ON Funding_Provider__c.Id = FP.Funding_Provider__c   
        LEFT JOIN salesforce_prod.Funding_Provider__c final ON final.Id = FP.FINAL_ASSET_OWNER__C 
        LEFT JOIN salesforce_prod.jrs_project JRSP ON M.Id = JRSP.ProjectId
        LEFT JOIN salesforce_prod.jrs_funding_product_attribute JRSFPA ON FP.Id = JRSFPA.Funding_Product__c
		LEFT JOIN salesforce_prod.loan_product__c LProd ON iQuote__c.ID = LProd.IQUOTE__C AND LProd.PRIMARY_PRODUCT__C = 1 AND LProd.ACTIVE__C = 1
		LEFT JOIN salesforce_prod.jrs_loan_product_attribute JRSLPA ON LProd.ID = JRSLPA.Loan_Product_Id 
		LEFT JOIN salesforce_prod.loan_provider__c LProv ON LProd.LOAN_PROVIDER__C = LProv.ID 
        
WHERE 
LProv.Name IN ('HERO','EFS','Admiral','CAL First')
AND M.IS_TEST__C = 0
AND M.STATUS__C <> 'Cancelled'
AND M.PERMIT_APPROVED_DATE__C IS NOT NULL
AND M.INSTALLATION_COMPLETE_DATE__C IS NOT NULL
AND M.INSTALLATION_COMPLETE_DATE_STATUS__C = 'Actual'
"""

In [10]:
sr2M1df = pd.read_sql_query(sr2M1query,conn)
sr2M1df['Week'] = 1

for i in range(len(sr2M1df.index)):
    if sr2M1df.loc[i, 'Day'] <= 7:
        sr2M1df.loc[i, 'Week'] = 1
    elif sr2M1df.loc[i,'Day'] <= 14 and sr2M1df.loc[i, 'Day'] > 7:
        sr2M1df.loc[i, 'Week'] = 2
    elif sr2M1df.loc[i,'Day'] <= 21 and sr2M1df.loc[i, 'Day'] > 14:
        sr2M1df.loc[i, 'Week'] = 3
    elif sr2M1df.loc[i,'Day'] <= 28 and sr2M1df.loc[i, 'Day'] > 21:
        sr2M1df.loc[i, 'Week'] = 4
    else:
        sr2M1df.loc[i, 'Week'] = 4

sr2M1df['Months_Week'] = sr2M1df.apply(lambda x: "{0} {1}".format(x['Months_to_M1'], x['Week']), axis = 1)
sr2M1df['Date'] = sr2M1df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
sr2M1df = pd.pivot_table(sr2M1df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
sr2M1df = sr2M1df.div(sr2M1df.iloc[:,-1], axis = 0)
sr2M1df = sr2M1df['Project Name']
sr2M1df = sr2M1df.reset_index()
sr2M1df.drop(sr2M1df.index[len(sr2M1df)-1])
sr2M1df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,1 1,1 2,1 3,1 4,2 1,2 2,2 3,...,4 3,4 4,5 1,5 2,5 3,5 4,6 1,6 2,6 3,6 4
0,Sale - M1,Sunrun 2,2016-04-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Sale - M1,Sunrun 2,2016-05-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Sale - M1,Sunrun 2,2016-10-01,0.000000,0.000000,0.000000,0.061303,0.000000,0.000000,0.000000,...,0.000000,0.168582,0.000000,0.000000,0.000000,0.053640,0.000000,0.000000,0.000000,0.026820
3,Sale - M1,Sunrun 2,2016-11-01,0.000000,0.033846,0.015385,0.000000,0.009231,0.236923,0.098462,...,0.036923,0.009231,0.006154,0.033846,0.009231,0.000000,0.003077,0.015385,0.012308,0.003077
4,Sale - M1,Sunrun 2,2016-12-01,0.013889,0.027778,0.027778,0.000000,0.183333,0.158333,0.119444,...,0.013889,0.002778,0.005556,0.002778,0.011111,0.000000,0.005556,0.005556,0.013889,0.000000
5,Sale - M1,Sunrun 2,2017-01-01,0.000000,0.000000,0.000000,0.000000,0.044776,0.044776,0.141791,...,0.067164,0.067164,0.007463,0.014925,0.007463,0.029851,0.007463,0.007463,0.014925,0.014925
6,Sale - M1,Sunrun 2,2017-02-01,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,...,0.000000,0.000000,0.095238,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000
7,All,,NaT,0.004448,0.018683,0.013345,0.014235,0.085409,0.124555,0.085409,...,0.023132,0.050712,0.008007,0.012456,0.007117,0.016014,0.005338,0.007117,0.009786,0.008897


In [11]:
sr2M2df = pd.read_sql_query(sr2M2query,conn)
sr2M2df['Week'] = 1

for i in range(len(sr2M2df.index)):
    if sr2M2df.loc[i, 'Day'] <= 7:
        sr2M2df.loc[i, 'Week'] = 1
    elif sr2M2df.loc[i,'Day'] <= 14 and sr2M2df.loc[i, 'Day'] > 7:
        sr2M2df.loc[i, 'Week'] = 2
    elif sr2M2df.loc[i,'Day'] <= 21 and sr2M2df.loc[i, 'Day'] > 14:
        sr2M2df.loc[i, 'Week'] = 3
    elif sr2M2df.loc[i,'Day'] <= 28 and sr2M2df.loc[i, 'Day'] > 21:
        sr2M2df.loc[i, 'Week'] = 4
    else:
        sr2M2df.loc[i, 'Week'] = 4

sr2M2df['Months_Week'] = sr2M2df.apply(lambda x: "{0} {1}".format(x['Months_M1_M2'], x['Week']), axis = 1)
sr2M2df['Date'] = sr2M2df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
sr2M2df = pd.pivot_table(sr2M2df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
sr2M2df = sr2M2df.div(sr2M2df.iloc[:,-1], axis = 0)
sr2M2df = sr2M2df['Project Name']
sr2M2df = sr2M2df.reset_index()
sr2M2df.drop(sr2M2df.index[len(sr2M2df)-1])
sr2M2df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,1 4,2 1,2 2,2 3,2 4
0,M1 - M2,Sunrun 2,2016-10-01,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M1 - M2,Sunrun 2,2016-11-01,0.525773,0.288660,0.082474,0.072165,0.000000,0.000000,0.000000,0.030928,0.000000,0.000000,0.000000,0.000000
2,M1 - M2,Sunrun 2,2016-12-01,0.488372,0.193798,0.197674,0.081395,0.003876,0.015504,0.017442,0.001938,0.000000,0.000000,0.000000,0.000000
3,M1 - M2,Sunrun 2,2017-01-01,0.181818,0.151515,0.066667,0.327273,0.042424,0.036364,0.054545,0.121212,0.000000,0.006061,0.006061,0.006061
4,M1 - M2,Sunrun 2,2017-02-01,0.615385,0.000000,0.000000,0.000000,0.307692,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000
5,All,,NaT,0.346918,0.152087,0.120278,0.303181,0.016899,0.013917,0.017893,0.023857,0.001988,0.000994,0.000994,0.000994


In [12]:
sr2M3df = pd.read_sql_query(sr2M3query,conn)
sr2M3df['Week'] = 1

for i in range(len(sr2M3df.index)):
    if sr2M3df.loc[i, 'Day'] <= 7:
        sr2M3df.loc[i, 'Week'] = 1
    elif sr2M3df.loc[i,'Day'] <= 14 and sr2M3df.loc[i, 'Day'] > 7:
        sr2M3df.loc[i, 'Week'] = 2
    elif sr2M3df.loc[i,'Day'] <= 21 and sr2M3df.loc[i, 'Day'] > 14:
        sr2M3df.loc[i, 'Week'] = 3
    elif sr2M3df.loc[i,'Day'] <= 28 and sr2M3df.loc[i, 'Day'] > 21:
        sr2M3df.loc[i, 'Week'] = 4
    else:
        sr2M3df.loc[i, 'Week'] = 4

sr2M3df['Months_Week'] = sr2M3df.apply(lambda x: "{0} {1}".format(x['Months_M2_M3'], x['Week']), axis = 1)
sr2M3df['Date'] = sr2M3df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
sr2M3df = pd.pivot_table(sr2M3df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
sr2M3df = sr2M3df.div(sr2M3df.iloc[:,-1], axis = 0)
sr2M3df = sr2M3df['Project Name']
sr2M3df = sr2M3df.reset_index()
sr2M3df.drop(sr2M3df.index[len(sr2M3df)-1])
sr2M3df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,1 4,2 2
0,M2 - M3,Sunrun 2,2016-11-01,0.568266,0.151292,0.158672,0.073801,0.000000,0.000000,0.000000,0.047970,0.000000
1,M2 - M3,Sunrun 2,2016-12-01,0.023739,0.661721,0.275964,0.000000,0.014837,0.011869,0.011869,0.000000,0.000000
2,M2 - M3,Sunrun 2,2017-01-01,0.349315,0.232877,0.133562,0.160959,0.003425,0.065068,0.023973,0.023973,0.006849
3,M2 - M3,Sunrun 2,2017-02-01,0.937500,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000
4,All,,NaT,0.315451,0.356223,0.187768,0.071888,0.008584,0.024678,0.011803,0.021459,0.002146


In [13]:
kinaM1df = pd.read_sql_query(kinaM1query,conn)
kinaM1df['Week'] = 1

for i in range(len(kinaM1df.index)):
    if kinaM1df.loc[i, 'Day'] <= 7:
        kinaM1df.loc[i, 'Week'] = 1
    elif kinaM1df.loc[i,'Day'] <= 14 and kinaM1df.loc[i, 'Day'] > 7:
        kinaM1df.loc[i, 'Week'] = 2
    elif kinaM1df.loc[i,'Day'] <= 21 and kinaM1df.loc[i, 'Day'] > 14:
        kinaM1df.loc[i, 'Week'] = 3
    elif kinaM1df.loc[i,'Day'] <= 28 and kinaM1df.loc[i, 'Day'] > 21:
        kinaM1df.loc[i, 'Week'] = 4
    else:
        kinaM1df.loc[i, 'Week'] = 4

kinaM1df['Months_Week'] = kinaM1df.apply(lambda x: "{0} {1}".format(x['Months_to_M1'], x['Week']), axis = 1)
kinaM1df['Date'] = kinaM1df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
kinaM1df = pd.pivot_table(kinaM1df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
kinaM1df = kinaM1df.div(kinaM1df.iloc[:,-1], axis = 0)
kinaM1df = kinaM1df['Project Name']
kinaM1df = kinaM1df.reset_index()
kinaM1df.drop(kinaM1df.index[len(kinaM1df)-1])
kinaM1df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,...,4 1,4 2,4 3,4 4,5 1,5 2,5 3,5 4,6 1,6 3
0,Sale - M1,Kinaole,2015-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057143,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Sale - M1,Kinaole,2016-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.040816,0.000000,...,0.000000,0.000000,0.000000,0.040816,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Sale - M1,Kinaole,2016-02-01,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.149254,...,0.000000,0.074627,0.074627,0.014925,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000
3,Sale - M1,Kinaole,2016-03-01,0.000000,0.009132,0.000000,0.000000,0.000000,0.242009,0.000000,...,0.000000,0.031963,0.000000,0.022831,0.000000,0.013699,0.000000,0.018265,0.000000,0.000000
4,Sale - M1,Kinaole,2016-04-01,0.000000,0.000000,0.034843,0.000000,0.177700,0.000000,0.324042,...,0.006969,0.000000,0.041812,0.000000,0.020906,0.000000,0.010453,0.000000,0.006969,0.010453
5,Sale - M1,Kinaole,2016-05-01,0.052632,0.000000,0.052632,0.000000,0.234818,0.000000,0.299595,...,0.012146,0.000000,0.020243,0.000000,0.004049,0.000000,0.020243,0.000000,0.004049,0.008097
6,Sale - M1,Kinaole,2016-06-01,0.013393,0.000000,0.049107,0.000000,0.285714,0.000000,0.303571,...,0.026786,0.000000,0.017857,0.000000,0.008929,0.000000,0.000000,0.000000,0.000000,0.008929
7,Sale - M1,Kinaole,2016-07-01,0.044118,0.000000,0.009804,0.004902,0.377451,0.000000,0.210784,...,0.009804,0.000000,0.009804,0.000000,0.000000,0.000000,0.004902,0.000000,0.009804,0.000000
8,Sale - M1,Kinaole,2016-08-01,0.016043,0.000000,0.000000,0.005348,0.342246,0.000000,0.000000,...,0.016043,0.000000,0.000000,0.010695,0.010695,0.000000,0.000000,0.000000,0.016043,0.000000
9,All,,NaT,0.018433,0.001317,0.024358,0.001317,0.206715,0.036208,0.190915,...,0.010533,0.007900,0.018433,0.006583,0.007242,0.001975,0.006583,0.002633,0.005267,0.004608


In [14]:
kinaM2df = pd.read_sql_query(kinaM2query,conn)
kinaM2df['Week'] = 1

for i in range(len(kinaM2df.index)):
    if kinaM2df.loc[i, 'Day'] <= 7:
        kinaM2df.loc[i, 'Week'] = 1
    elif kinaM2df.loc[i,'Day'] <= 14 and kinaM2df.loc[i, 'Day'] > 7:
        kinaM2df.loc[i, 'Week'] = 2
    elif kinaM2df.loc[i,'Day'] <= 21 and kinaM2df.loc[i, 'Day'] > 14:
        kinaM2df.loc[i, 'Week'] = 3
    elif kinaM2df.loc[i,'Day'] <= 28 and kinaM2df.loc[i, 'Day'] > 21:
        kinaM2df.loc[i, 'Week'] = 4
    else:
        kinaM2df.loc[i, 'Week'] = 4

kinaM2df['Months_Week'] = kinaM2df.apply(lambda x: "{0} {1}".format(x['Months_M1_M2'], x['Week']), axis = 1)
kinaM2df['Date'] = kinaM2df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
kinaM2df = pd.pivot_table(kinaM2df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
kinaM2df = kinaM2df.div(kinaM2df.iloc[:,-1], axis = 0)
kinaM2df = kinaM2df['Project Name']
kinaM2df = kinaM2df.reset_index()
kinaM2df.drop(kinaM2df.index[len(kinaM2df)-1])
kinaM2df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,...,3 3,3 4,4 1,4 3,4 4,5 2,5 4,6 1,6 2,6 3
0,M1 - M2,Kinaole,2015-12-01,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M1 - M2,Kinaole,2016-01-01,0.000000,0.535714,0.000000,0.214286,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,M1 - M2,Kinaole,2016-02-01,0.000000,0.444444,0.185185,0.000000,0.000000,0.111111,0.111111,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M1 - M2,Kinaole,2016-03-01,0.000000,0.133333,0.000000,0.266667,0.000000,0.253333,0.000000,...,0.000000,0.013333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,M1 - M2,Kinaole,2016-04-01,0.418301,0.000000,0.294118,0.000000,0.058824,0.000000,0.143791,...,0.006536,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M1 - M2,Kinaole,2016-05-01,0.259109,0.000000,0.404858,0.000000,0.129555,0.000000,0.133603,...,0.016194,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M1 - M2,Kinaole,2016-06-01,0.131387,0.000000,0.113139,0.000000,0.306569,0.000000,0.240876,...,0.014599,0.000000,0.000000,0.007299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,M1 - M2,Kinaole,2016-07-01,0.139098,0.000000,0.334586,0.000000,0.206767,0.000000,0.078947,...,0.060150,0.000000,0.000000,0.003759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M1 - M2,Kinaole,2016-08-01,0.175182,0.000000,0.000000,0.259124,0.065693,0.000000,0.000000,...,0.000000,0.021898,0.025547,0.000000,0.010949,0.000000,0.000000,0.000000,0.000000,0.000000
9,M1 - M2,Kinaole,2016-09-01,0.258741,0.000000,0.000000,0.000000,0.279720,0.000000,0.000000,...,0.000000,0.020979,0.006993,0.000000,0.006993,0.000000,0.020979,0.000000,0.000000,0.000000


In [15]:
kinaM3df = pd.read_sql_query(kinaM3query,conn)
kinaM3df['Week'] = 1

for i in range(len(kinaM3df.index)):
    if kinaM3df.loc[i, 'Day'] <= 7:
        kinaM3df.loc[i, 'Week'] = 1
    elif kinaM3df.loc[i,'Day'] <= 14 and kinaM3df.loc[i, 'Day'] > 7:
        kinaM3df.loc[i, 'Week'] = 2
    elif kinaM3df.loc[i,'Day'] <= 21 and kinaM3df.loc[i, 'Day'] > 14:
        kinaM3df.loc[i, 'Week'] = 3
    elif kinaM3df.loc[i,'Day'] <= 28 and kinaM3df.loc[i, 'Day'] > 21:
        kinaM3df.loc[i, 'Week'] = 4
    else:
        kinaM3df.loc[i, 'Week'] = 4

kinaM3df['Months_Week'] = kinaM3df.apply(lambda x: "{0} {1}".format(x['Months_M2_M3'], x['Week']), axis = 1)
kinaM3df['Date'] = kinaM3df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
kinaM3df = pd.pivot_table(kinaM3df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
kinaM3df = kinaM3df.div(kinaM3df.iloc[:,-1], axis = 0)
kinaM3df = kinaM3df['Project Name']
kinaM3df = kinaM3df.reset_index()
kinaM3df.drop(kinaM3df.index[len(kinaM3df)-1])
kinaM3df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,...,2 2,2 3,2 4,3 1,3 2,3 3,3 4,4 1,4 2,5 2
0,M2 - M3,Kinaole,2016-01-01,0.000000,0.823529,0.000000,0.176471,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M2 - M3,Kinaole,2016-02-01,0.000000,0.368421,0.000000,0.578947,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,M2 - M3,Kinaole,2016-03-01,0.000000,0.417910,0.000000,0.447761,0.000000,0.134328,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M2 - M3,Kinaole,2016-04-01,0.403727,0.000000,0.385093,0.192547,0.000000,0.000000,0.000000,...,0.000000,0.006211,0.006211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,M2 - M3,Kinaole,2016-05-01,0.250000,0.000000,0.732955,0.000000,0.011364,0.000000,0.005682,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M2 - M3,Kinaole,2016-06-01,0.388889,0.000000,0.452381,0.000000,0.043651,0.000000,0.099206,...,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M2 - M3,Kinaole,2016-07-01,0.366534,0.000000,0.529880,0.000000,0.071713,0.000000,0.003984,...,0.000000,0.019920,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,M2 - M3,Kinaole,2016-08-01,0.424561,0.000000,0.000000,0.442105,0.024561,0.000000,0.000000,...,0.000000,0.000000,0.010526,0.007018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M2 - M3,Kinaole,2016-09-01,0.502591,0.000000,0.000000,0.336788,0.072539,0.000000,0.000000,...,0.000000,0.000000,0.010363,0.005181,0.000000,0.000000,0.005181,0.000000,0.000000,0.000000
9,M2 - M3,Kinaole,2016-10-01,0.600000,0.000000,0.185714,0.000000,0.100000,0.000000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.000000,0.000000


In [16]:
mosaicM1df = pd.read_sql_query(mosaicM1query,conn)
mosaicM1df['Week'] = 1

for i in range(len(mosaicM1df.index)):
    if mosaicM1df.loc[i, 'Day'] <= 7:
        mosaicM1df.loc[i, 'Week'] = 1
    elif mosaicM1df.loc[i,'Day'] <= 14 and mosaicM1df.loc[i, 'Day'] > 7:
        mosaicM1df.loc[i, 'Week'] = 2
    elif mosaicM1df.loc[i,'Day'] <= 21 and mosaicM1df.loc[i, 'Day'] > 14:
        mosaicM1df.loc[i, 'Week'] = 3
    elif mosaicM1df.loc[i,'Day'] <= 28 and mosaicM1df.loc[i, 'Day'] > 21:
        mosaicM1df.loc[i, 'Week'] = 4
    else:
        mosaicM1df.loc[i, 'Week'] = 4

mosaicM1df['Months_Week'] = mosaicM1df.apply(lambda x: "{0} {1}".format(x['Months_to_M1'], x['Week']), axis = 1)
mosaicM1df['Date'] = mosaicM1df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
mosaicM1df = pd.pivot_table(mosaicM1df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
mosaicM1df = mosaicM1df.div(mosaicM1df.iloc[:,-1], axis = 0)
mosaicM1df = mosaicM1df['Project Name']
mosaicM1df = mosaicM1df.reset_index()
mosaicM1df.drop(mosaicM1df.index[len(mosaicM1df)-1])
mosaicM1df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,...,4 3,4 4,5 1,5 2,5 3,5 4,6 1,6 2,6 3,6 4
0,Sale - M1,Mosaic,2015-08-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Sale - M1,Mosaic,2015-09-01,0.050000,0.025000,0.050000,0.050000,0.050000,0.150000,0.300000,...,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Sale - M1,Mosaic,2015-10-01,0.000000,0.000000,0.038462,0.076923,0.076923,0.134615,0.153846,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000
3,Sale - M1,Mosaic,2015-11-01,0.000000,0.000000,0.000000,0.000000,0.177778,0.133333,0.133333,...,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000
4,Sale - M1,Mosaic,2015-12-01,0.000000,0.000000,0.000000,0.000000,0.078431,0.078431,0.039216,...,0.000000,0.039216,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Sale - M1,Mosaic,2016-01-01,0.000000,0.000000,0.023256,0.000000,0.023256,0.023256,0.023256,...,0.046512,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000
6,Sale - M1,Mosaic,2016-02-01,0.013514,0.000000,0.013514,0.027027,0.013514,0.013514,0.027027,...,0.013514,0.067568,0.040541,0.000000,0.027027,0.013514,0.013514,0.000000,0.000000,0.027027
7,Sale - M1,Mosaic,2016-03-01,0.024691,0.049383,0.037037,0.037037,0.049383,0.086420,0.111111,...,0.012346,0.037037,0.000000,0.000000,0.012346,0.037037,0.000000,0.012346,0.000000,0.037037
8,Sale - M1,Mosaic,2016-04-01,0.028302,0.047170,0.047170,0.075472,0.103774,0.084906,0.179245,...,0.018868,0.018868,0.018868,0.028302,0.018868,0.009434,0.009434,0.000000,0.000000,0.009434
9,Sale - M1,Mosaic,2016-05-01,0.043210,0.030864,0.037037,0.043210,0.098765,0.074074,0.086420,...,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.012346,0.006173,0.006173,0.006173


In [17]:
mosaicM2df = pd.read_sql_query(mosaicM2query,conn)
mosaicM2df['Week'] = 1

for i in range(len(mosaicM2df.index)):
    if mosaicM2df.loc[i, 'Day'] <= 7:
        mosaicM2df.loc[i, 'Week'] = 1
    elif mosaicM2df.loc[i,'Day'] <= 14 and mosaicM2df.loc[i, 'Day'] > 7:
        mosaicM2df.loc[i, 'Week'] = 2
    elif mosaicM2df.loc[i,'Day'] <= 21 and mosaicM2df.loc[i, 'Day'] > 14:
        mosaicM2df.loc[i, 'Week'] = 3
    elif mosaicM2df.loc[i,'Day'] <= 28 and mosaicM2df.loc[i, 'Day'] > 21:
        mosaicM2df.loc[i, 'Week'] = 4
    else:
        mosaicM2df.loc[i, 'Week'] = 4

mosaicM2df['Months_Week'] = mosaicM2df.apply(lambda x: "{0} {1}".format(x['Months_M1_M2'], x['Week']), axis = 1)
mosaicM2df['Date'] = mosaicM2df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
mosaicM2df = pd.pivot_table(mosaicM2df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
mosaicM2df = mosaicM2df.div(mosaicM2df.iloc[:,-1], axis = 0)
mosaicM2df = mosaicM2df['Project Name']
mosaicM2df = mosaicM2df.reset_index()
mosaicM2df.drop(mosaicM2df.index[len(mosaicM2df)-1])
mosaicM2df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,...,3 2,3 3,3 4,4 2,4 3,4 4,5 1,5 2,6 2,6 4
0,M1 - M2,Mosaic,2015-09-01,0.187500,0.187500,0.312500,0.312500,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M1 - M2,Mosaic,2015-10-01,0.068966,0.172414,0.310345,0.241379,0.000000,0.068966,0.068966,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,M1 - M2,Mosaic,2015-11-01,0.152174,0.130435,0.152174,0.108696,0.021739,0.130435,0.130435,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M1 - M2,Mosaic,2015-12-01,0.050000,0.150000,0.175000,0.175000,0.100000,0.125000,0.075000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,M1 - M2,Mosaic,2016-01-01,0.108696,0.086957,0.130435,0.130435,0.152174,0.108696,0.065217,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M1 - M2,Mosaic,2016-02-01,0.105263,0.105263,0.087719,0.192982,0.070175,0.052632,0.070175,...,0.000000,0.035088,0.000000,0.000000,0.017544,0.017544,0.000000,0.000000,0.000000,0.000000
6,M1 - M2,Mosaic,2016-03-01,0.109890,0.076923,0.087912,0.373626,0.043956,0.065934,0.065934,...,0.010989,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989
7,M1 - M2,Mosaic,2016-04-01,0.158537,0.097561,0.207317,0.256098,0.073171,0.060976,0.036585,...,0.000000,0.000000,0.012195,0.000000,0.000000,0.000000,0.012195,0.000000,0.000000,0.000000
8,M1 - M2,Mosaic,2016-05-01,0.220339,0.161017,0.169492,0.186441,0.050847,0.076271,0.016949,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M1 - M2,Mosaic,2016-06-01,0.118012,0.285714,0.124224,0.167702,0.055901,0.055901,0.049689,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006211,0.006211,0.000000,0.000000


In [18]:
mosaicM3df = pd.read_sql_query(mosaicM3query,conn)
mosaicM3df['Week'] = 1

for i in range(len(mosaicM3df.index)):
    if mosaicM3df.loc[i, 'Day'] <= 7:
        mosaicM3df.loc[i, 'Week'] = 1
    elif mosaicM3df.loc[i,'Day'] <= 14 and mosaicM3df.loc[i, 'Day'] > 7:
        mosaicM3df.loc[i, 'Week'] = 2
    elif mosaicM3df.loc[i,'Day'] <= 21 and mosaicM3df.loc[i, 'Day'] > 14:
        mosaicM3df.loc[i, 'Week'] = 3
    elif mosaicM3df.loc[i,'Day'] <= 28 and mosaicM3df.loc[i, 'Day'] > 21:
        mosaicM3df.loc[i, 'Week'] = 4
    else:
        mosaicM3df.loc[i, 'Week'] = 4

mosaicM3df['Months_Week'] = mosaicM3df.apply(lambda x: "{0} {1}".format(x['Months_M2_M3'], x['Week']), axis = 1)
mosaicM3df['Date'] = mosaicM3df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
mosaicM3df = pd.pivot_table(mosaicM3df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
mosaicM3df = mosaicM3df.div(mosaicM3df.iloc[:,-1], axis = 0)
mosaicM3df = mosaicM3df['Project Name']
mosaicM3df = mosaicM3df.reset_index()
mosaicM3df.drop(mosaicM3df.index[len(mosaicM3df)-1])
mosaicM3df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,...,2 4,3 1,3 2,3 3,3 4,4 1,4 4,5 2,5 4,6 4
0,M2 - M3,Mosaic,2015-09-01,0.000000,0.000000,0.333333,0.666667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M2 - M3,Mosaic,2015-10-01,0.148148,0.148148,0.222222,0.444444,0.000000,0.000000,0.037037,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,M2 - M3,Mosaic,2015-11-01,0.322581,0.096774,0.290323,0.258065,0.032258,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M2 - M3,Mosaic,2015-12-01,0.244444,0.222222,0.244444,0.155556,0.000000,0.022222,0.022222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,M2 - M3,Mosaic,2016-01-01,0.277778,0.240741,0.166667,0.240741,0.000000,0.037037,0.000000,...,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M2 - M3,Mosaic,2016-02-01,0.166667,0.277778,0.185185,0.240741,0.000000,0.055556,0.018519,...,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M2 - M3,Mosaic,2016-03-01,0.164557,0.177215,0.202532,0.455696,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,M2 - M3,Mosaic,2016-04-01,0.153846,0.340659,0.197802,0.285714,0.000000,0.010989,0.010989,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M2 - M3,Mosaic,2016-05-01,0.239583,0.302083,0.187500,0.250000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M2 - M3,Mosaic,2016-06-01,0.126582,0.234177,0.310127,0.259494,0.006329,0.031646,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
purecashM1df = pd.read_sql_query(purecashM1query,conn)
purecashM1df['Week'] = 1

for i in range(len(purecashM1df.index)):
    if purecashM1df.loc[i, 'Day'] <= 7:
        purecashM1df.loc[i, 'Week'] = 1
    elif purecashM1df.loc[i,'Day'] <= 14 and purecashM1df.loc[i, 'Day'] > 7:
        purecashM1df.loc[i, 'Week'] = 2
    elif purecashM1df.loc[i,'Day'] <= 21 and purecashM1df.loc[i, 'Day'] > 14:
        purecashM1df.loc[i, 'Week'] = 3
    elif purecashM1df.loc[i,'Day'] <= 28 and purecashM1df.loc[i, 'Day'] > 21:
        purecashM1df.loc[i, 'Week'] = 4
    else:
        purecashM1df.loc[i, 'Week'] = 4

purecashM1df['Months_Week'] = purecashM1df.apply(lambda x: "{0} {1}".format(x['Months_to_M1'], x['Week']), axis = 1)
purecashM1df['Date'] = purecashM1df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
purecashM1df = pd.pivot_table(purecashM1df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
purecashM1df = purecashM1df.div(purecashM1df.iloc[:,-1], axis = 0)
purecashM1df = purecashM1df['Project Name']
purecashM1df = purecashM1df.reset_index()
purecashM1df.drop(purecashM1df.index[len(purecashM1df)-1])
purecashM1df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4
0,Sale - M1,Pure Cash,2012-12-01,0.000000,1.000000,0.000000,0.000000
1,Sale - M1,Pure Cash,2013-01-01,0.000000,0.000000,0.333333,0.666667
2,Sale - M1,Pure Cash,2013-03-01,0.000000,0.400000,0.000000,0.600000
3,Sale - M1,Pure Cash,2013-04-01,0.000000,0.500000,0.000000,0.500000
4,Sale - M1,Pure Cash,2013-05-01,0.000000,0.000000,0.250000,0.750000
5,Sale - M1,Pure Cash,2013-06-01,0.250000,0.250000,0.000000,0.500000
6,Sale - M1,Pure Cash,2013-07-01,0.200000,0.000000,0.200000,0.600000
7,Sale - M1,Pure Cash,2013-08-01,0.000000,0.000000,1.000000,0.000000
8,Sale - M1,Pure Cash,2013-09-01,0.500000,0.166667,0.000000,0.333333
9,Sale - M1,Pure Cash,2013-10-01,0.000000,0.500000,0.000000,0.500000


In [20]:
purecashM2df = pd.read_sql_query(purecashM2query,conn)
purecashM2df['Week'] = 1

for i in range(len(purecashM2df.index)):
    if purecashM2df.loc[i, 'Day'] <= 7:
        purecashM2df.loc[i, 'Week'] = 1
    elif purecashM2df.loc[i,'Day'] <= 14 and purecashM2df.loc[i, 'Day'] > 7:
        purecashM2df.loc[i, 'Week'] = 2
    elif purecashM2df.loc[i,'Day'] <= 21 and purecashM2df.loc[i, 'Day'] > 14:
        purecashM2df.loc[i, 'Week'] = 3
    elif purecashM2df.loc[i,'Day'] <= 28 and purecashM2df.loc[i, 'Day'] > 21:
        purecashM2df.loc[i, 'Week'] = 4
    else:
        purecashM2df.loc[i, 'Week'] = 4

purecashM2df['Months_Week'] = purecashM2df.apply(lambda x: "{0} {1}".format(x['Months_M1_M2'], x['Week']), axis = 1)
purecashM2df['Date'] = purecashM2df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
purecashM2df = pd.pivot_table(purecashM2df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
purecashM2df = purecashM2df.div(purecashM2df.iloc[:,-1], axis = 0)
purecashM2df = purecashM2df['Project Name']
purecashM2df = purecashM2df.reset_index()
purecashM2df.drop(purecashM2df.index[len(purecashM2df)-1])
purecashM2df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,...,4 2,4 3,4 4,5 2,5 3,5 4,6 1,6 2,6 3,6 4
0,M1 - M2,Pure Cash,2013-04-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000
1,M1 - M2,Pure Cash,2013-06-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000
2,M1 - M2,Pure Cash,2013-07-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000
3,M1 - M2,Pure Cash,2013-08-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000
4,M1 - M2,Pure Cash,2013-09-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00000,0.00000,0.000000,0.00000
5,M1 - M2,Pure Cash,2013-10-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.25000,0.000000,0.00000
6,M1 - M2,Pure Cash,2013-11-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.200000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000
7,M1 - M2,Pure Cash,2013-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000
8,M1 - M2,Pure Cash,2014-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000
9,M1 - M2,Pure Cash,2014-02-01,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.166667,0.00000,0.00000,0.000000,0.00000


In [21]:
purecashM3df = pd.read_sql_query(purecashM3query,conn)
purecashM3df['Week'] = 1

for i in range(len(purecashM3df.index)):
    if purecashM3df.loc[i, 'Day'] <= 7:
        purecashM3df.loc[i, 'Week'] = 1
    elif purecashM3df.loc[i,'Day'] <= 14 and purecashM3df.loc[i, 'Day'] > 7:
        purecashM3df.loc[i, 'Week'] = 2
    elif purecashM3df.loc[i,'Day'] <= 21 and purecashM3df.loc[i, 'Day'] > 14:
        purecashM3df.loc[i, 'Week'] = 3
    elif purecashM3df.loc[i,'Day'] <= 28 and purecashM3df.loc[i, 'Day'] > 21:
        purecashM3df.loc[i, 'Week'] = 4
    else:
        purecashM3df.loc[i, 'Week'] = 4

purecashM3df['Months_Week'] = purecashM3df.apply(lambda x: "{0} {1}".format(x['Months_M2_M3'], x['Week']), axis = 1)
purecashM3df['Date'] = purecashM3df.apply(lambda x:datetime.strptime("{0} {1}".format(x['Month'],x['Year']), "%m %Y"), axis = 1)
purecashM3df = pd.pivot_table(purecashM3df, index = ['Milestone','Fund','Date'], values = ['Project Name'], columns = ['Months_Week'], aggfunc = np.count_nonzero, fill_value = 0, margins = True)
purecashM3df = purecashM3df.div(purecashM3df.iloc[:,-1], axis = 0)
purecashM3df = purecashM3df['Project Name']
purecashM3df = purecashM3df.reset_index()
purecashM3df.drop(purecashM3df.index[len(purecashM3df)-1])
purecashM3df.drop('All',axis =1)

Months_Week,Milestone,Fund,Date,0 1,0 2,0 3,0 4,1 1,1 2,1 3,...,2 1,2 2,2 3,2 4,3 1,3 2,3 4,6 1,6 2,6 4
0,M2 - M3,Pure Cash,2013-12-01,0.666667,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M2 - M3,Pure Cash,2014-01-01,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,M2 - M3,Pure Cash,2014-02-01,0.250000,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M2 - M3,Pure Cash,2014-03-01,0.000000,0.333333,0.000000,0.666667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,M2 - M3,Pure Cash,2014-04-01,0.125000,0.125000,0.000000,0.250000,0.250000,0.125000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M2 - M3,Pure Cash,2014-05-01,0.200000,0.200000,0.000000,0.200000,0.000000,0.200000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M2 - M3,Pure Cash,2014-06-01,0.142857,0.000000,0.000000,0.142857,0.285714,0.142857,0.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,M2 - M3,Pure Cash,2014-07-01,0.200000,0.200000,0.000000,0.400000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M2 - M3,Pure Cash,2014-08-01,0.250000,0.250000,0.000000,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M2 - M3,Pure Cash,2014-09-01,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
milestone_velocity = pd.concat([kinaM1df, kinaM2df, kinaM3df, mosaicM1df, mosaicM2df, mosaicM3df, purecashM1df, purecashM2df, purecashM3df, sr2M1df, sr2M2df, sr2M3df],
                               axis = 0, ignore_index=True)


In [23]:
milestone_velocity = milestone_velocity[milestone_velocity.Milestone != 'All']
cols = ['Fund', 'Milestone','Date', '6 4','6 3','6 2','6 1','5 4', '5 3','5 2','5 1','4 4','4 3','4 2','4 1','3 4','3 3','3 2','3 1','2 4','2 3','2 2','2 1','1 4','1 3',
        '1 2','1 1','0 4','0 3','0 2','0 1']
milestone_velocity = milestone_velocity[cols]
milestone_velocity.columns = ['Fund','Milestone','Date'
                              ,'6_Months_Week4_Previous', '6_Months_Week3_Previous', '6_Months_Week2_Previous', '6_Months_Week1_Previous'
                              ,'5_Months_Week4_Previous', '5_Months_Week3_Previous', '5_Months_Week2_Previous', '5_Months_Week1_Previous'
                              ,'4_Months_Week4_Previous', '4_Months_Week3_Previous', '4_Months_Week2_Previous', '4_Months_Week1_Previous'
                              ,'3_Months_Week4_Previous', '3_Months_Week3_Previous', '3_Months_Week2_Previous', '3_Months_Week1_Previous'
                              ,'2_Months_Week4_Previous', '2_Months_Week3_Previous', '2_Months_Week2_Previous', '2_Months_Week1_Previous'
                              ,'1_Months_Week4_Previous', '1_Months_Week3_Previous', '1_Months_Week2_Previous', '1_Months_Week1_Previous'
                             ,'0_Months_Week4_Previous', '0_Months_Week3_Previous', '0_Months_Week2_Previous', '0_Months_Week1_Previous']

milestone_velocity.fillna('0.000000')

,Fund,Milestone,Date,6_Months_Week4_Previous,6_Months_Week3_Previous,6_Months_Week2_Previous,6_Months_Week1_Previous,5_Months_Week4_Previous,5_Months_Week3_Previous,5_Months_Week2_Previous,...,2_Months_Week2_Previous,2_Months_Week1_Previous,1_Months_Week4_Previous,1_Months_Week3_Previous,1_Months_Week2_Previous,1_Months_Week1_Previous,0_Months_Week4_Previous,0_Months_Week3_Previous,0_Months_Week2_Previous,0_Months_Week1_Previous
0,Kinaole,Sale - M1,2015-12-01,0.000000,0,0.000000,0,0,0,0,...,0,0,0.0857143,0.0571429,0,0,0,0,0,0
1,Kinaole,Sale - M1,2016-01-01,0.000000,0,0.000000,0,0,0,0,...,0.265306,0,0.0204082,0,0.0408163,0,0,0,0,0
2,Kinaole,Sale - M1,2016-02-01,0.000000,0,0.000000,0,0,0.0149254,0,...,0.0597015,0,0,0.149254,0,0,0,0.0149254,0,0
3,Kinaole,Sale - M1,2016-03-01,0.000000,0,0.000000,0,0.0182648,0,0.0136986,...,0.136986,0,0.292237,0,0.242009,0,0,0,0.00913242,0
4,Kinaole,Sale - M1,2016-04-01,0.000000,0.010453,0.000000,0.00696864,0,0.010453,0,...,0,0.0940767,0,0.324042,0,0.1777,0,0.0348432,0,0
5,Kinaole,Sale - M1,2016-05-01,0.000000,0.00809717,0.000000,0.00404858,0,0.0202429,0,...,0,0.0850202,0,0.299595,0,0.234818,0,0.0526316,0,0.0526316
6,Kinaole,Sale - M1,2016-06-01,0.000000,0.00892857,0.000000,0,0,0,0,...,0,0.129464,0,0.303571,0,0.285714,0,0.0491071,0,0.0133929
7,Kinaole,Sale - M1,2016-07-01,0.000000,0,0.000000,0.00980392,0,0.00490196,0,...,0,0.132353,0,0.210784,0,0.377451,0.00490196,0.00980392,0,0.0441176
8,Kinaole,Sale - M1,2016-08-01,0.000000,0,0.000000,0.0160428,0,0,0,...,0,0.208556,0.187166,0,0,0.342246,0.00534759,0,0,0.0160428
10,Kinaole,M1 - M2,2015-12-01,0.000000,0,0,0,0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0


In [24]:
for i in range(len(milestone_velocity.index)):
    milestone_velocity.iloc[i,3] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,3]))
    milestone_velocity.iloc[i,4] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,4]))
    milestone_velocity.iloc[i,5] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,5]))
    milestone_velocity.iloc[i,6] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,6]))
    milestone_velocity.iloc[i,7] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,7]))
    milestone_velocity.iloc[i,8] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,8]))
    milestone_velocity.iloc[i,9] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,9]))
    milestone_velocity.iloc[i,10] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,10]))
    milestone_velocity.iloc[i,11] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,11]))
    milestone_velocity.iloc[i,12] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,12]))
    milestone_velocity.iloc[i,13] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,13]))
    milestone_velocity.iloc[i,14] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,14]))
    milestone_velocity.iloc[i,15] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,15]))
    milestone_velocity.iloc[i,16] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,16]))
    milestone_velocity.iloc[i,17] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,17]))
    milestone_velocity.iloc[i,18] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,18]))
    milestone_velocity.iloc[i,19] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,19]))
    milestone_velocity.iloc[i,20] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,20]))
    milestone_velocity.iloc[i,21] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,21]))
    milestone_velocity.iloc[i,22] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,22]))
    milestone_velocity.iloc[i,23] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,23]))
    milestone_velocity.iloc[i,24] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,24]))
    milestone_velocity.iloc[i,25] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,25]))
    milestone_velocity.iloc[i,26] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,26]))
    milestone_velocity.iloc[i,27] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,27]))
    milestone_velocity.iloc[i,28] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,28]))
    milestone_velocity.iloc[i,29] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,29]))
    milestone_velocity.iloc[i,30] =  "{0:.6f}".format(float(milestone_velocity.iloc[i,30]))
    
    


In [25]:
for i in range(len(milestone_velocity.index)):
    if milestone_velocity.iloc[i,3] == 'nan':
        milestone_velocity.iloc[i,3] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,4] == 'nan':
        milestone_velocity.iloc[i,4] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,5] == 'nan':
        milestone_velocity.iloc[i,5] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,6] == 'nan':
        milestone_velocity.iloc[i,6] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,7] == 'nan':
        milestone_velocity.iloc[i,7] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,8] == 'nan':
        milestone_velocity.iloc[i,8] = "{0:.6f}".format(0)       
    if milestone_velocity.iloc[i,9] == 'nan':
        milestone_velocity.iloc[i,9] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,10] == 'nan':
        milestone_velocity.iloc[i,10] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,11] == 'nan':
        milestone_velocity.iloc[i,11] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,12] == 'nan':
        milestone_velocity.iloc[i,12] = "{0:.6f}".format(0)   
    if milestone_velocity.iloc[i,13] == 'nan':
        milestone_velocity.iloc[i,13] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,14] == 'nan':
        milestone_velocity.iloc[i,14] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,15] == 'nan':
        milestone_velocity.iloc[i,15] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,16] == 'nan':
        milestone_velocity.iloc[i,16] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,17] == 'nan':
        milestone_velocity.iloc[i,17] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,18] == 'nan':
        milestone_velocity.iloc[i,18] = "{0:.6f}".format(0)       
    if milestone_velocity.iloc[i,19] == 'nan':
        milestone_velocity.iloc[i,19] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,20] == 'nan':
        milestone_velocity.iloc[i,20] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,21] == 'nan':
        milestone_velocity.iloc[i,21] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,22] == 'nan':
        milestone_velocity.iloc[i,22] = "{0:.6f}".format(0)   
    if milestone_velocity.iloc[i,23] == 'nan':
        milestone_velocity.iloc[i,23] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,24] == 'nan':
        milestone_velocity.iloc[i,24] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,25] == 'nan':
        milestone_velocity.iloc[i,25] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,26] == 'nan':
        milestone_velocity.iloc[i,26] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,27] == 'nan':
        milestone_velocity.iloc[i,27] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,28] == 'nan':
        milestone_velocity.iloc[i,28] = "{0:.6f}".format(0)       
    if milestone_velocity.iloc[i,29] == 'nan':
        milestone_velocity.iloc[i,29] = "{0:.6f}".format(0)
    if milestone_velocity.iloc[i,30] == 'nan':
        milestone_velocity.iloc[i,30] = "{0:.6f}".format(0)

In [26]:
milestone_velocity['Date'] = [time.date() for time in milestone_velocity['Date']]
milestone_velocity['Last_Refreshed_Date'] = today

In [27]:
cols = list(milestone_velocity)
cols.insert(0,cols.pop(cols.index('Date')))
milestone_velocity = milestone_velocity.ix[:,cols]

In [28]:
milestone_velocity

,Date,Fund,Milestone,6_Months_Week4_Previous,6_Months_Week3_Previous,6_Months_Week2_Previous,6_Months_Week1_Previous,5_Months_Week4_Previous,5_Months_Week3_Previous,5_Months_Week2_Previous,...,2_Months_Week1_Previous,1_Months_Week4_Previous,1_Months_Week3_Previous,1_Months_Week2_Previous,1_Months_Week1_Previous,0_Months_Week4_Previous,0_Months_Week3_Previous,0_Months_Week2_Previous,0_Months_Week1_Previous,Last_Refreshed_Date
0,2015-12-01,Kinaole,Sale - M1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.085714,0.057143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2017-02-06
1,2016-01-01,Kinaole,Sale - M1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.020408,0.000000,0.040816,0.000000,0.000000,0.000000,0.000000,0.000000,2017-02-06
2,2016-02-01,Kinaole,Sale - M1,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,...,0.000000,0.000000,0.149254,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,2017-02-06
3,2016-03-01,Kinaole,Sale - M1,0.000000,0.000000,0.000000,0.000000,0.018265,0.000000,0.013699,...,0.000000,0.292237,0.000000,0.242009,0.000000,0.000000,0.000000,0.009132,0.000000,2017-02-06
4,2016-04-01,Kinaole,Sale - M1,0.000000,0.010453,0.000000,0.006969,0.000000,0.010453,0.000000,...,0.094077,0.000000,0.324042,0.000000,0.177700,0.000000,0.034843,0.000000,0.000000,2017-02-06
5,2016-05-01,Kinaole,Sale - M1,0.000000,0.008097,0.000000,0.004049,0.000000,0.020243,0.000000,...,0.085020,0.000000,0.299595,0.000000,0.234818,0.000000,0.052632,0.000000,0.052632,2017-02-06
6,2016-06-01,Kinaole,Sale - M1,0.000000,0.008929,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.129464,0.000000,0.303571,0.000000,0.285714,0.000000,0.049107,0.000000,0.013393,2017-02-06
7,2016-07-01,Kinaole,Sale - M1,0.000000,0.000000,0.000000,0.009804,0.000000,0.004902,0.000000,...,0.132353,0.000000,0.210784,0.000000,0.377451,0.004902,0.009804,0.000000,0.044118,2017-02-06
8,2016-08-01,Kinaole,Sale - M1,0.000000,0.000000,0.000000,0.016043,0.000000,0.000000,0.000000,...,0.208556,0.187166,0.000000,0.000000,0.342246,0.005348,0.000000,0.000000,0.016043,2017-02-06
10,2015-12-01,Kinaole,M1 - M2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2017-02-06


In [5]:
milestone_velocity.to_sql('milestone_velocity_reverse_weekly', conn, flavor = 'mysql', if_exists = 'replace', index = False)

NameError: name 'milestone_velocity' is not defined

In [6]:
alter_table_query = """
ALTER TABLE fiops.milestone_velocity_reverse_weekly
MODIFY 6_Months_Week4_Previous decimal(7,6),
MODIFY 6_Months_Week3_Previous decimal(7,6),
MODIFY 6_Months_Week2_Previous decimal(7,6),
MODIFY 6_Months_Week1_Previous decimal(7,6),
MODIFY 5_Months_Week4_Previous decimal(7,6),
MODIFY 5_Months_Week3_Previous decimal(7,6),
MODIFY 5_Months_Week2_Previous decimal(7,6),
MODIFY 5_Months_Week1_Previous decimal(7,6),
MODIFY 4_Months_Week4_Previous decimal(7,6),
MODIFY 4_Months_Week3_Previous decimal(7,6),
MODIFY 4_Months_Week2_Previous decimal(7,6),
MODIFY 4_Months_Week1_Previous decimal(7,6),
MODIFY 3_Months_Week4_Previous decimal(7,6),
MODIFY 3_Months_Week3_Previous decimal(7,6),
MODIFY 3_Months_Week2_Previous decimal(7,6),
MODIFY 3_Months_Week1_Previous decimal(7,6),
MODIFY 2_Months_Week4_Previous decimal(7,6),
MODIFY 2_Months_Week3_Previous decimal(7,6),
MODIFY 2_Months_Week2_Previous decimal(7,6),
MODIFY 2_Months_Week1_Previous decimal(7,6),
MODIFY 1_Months_Week4_Previous decimal(7,6),
MODIFY 1_Months_Week3_Previous decimal(7,6),
MODIFY 1_Months_Week2_Previous decimal(7,6),
MODIFY 1_Months_Week1_Previous decimal(7,6),
MODIFY 0_Months_Week4_Previous decimal(7,6),
MODIFY 0_Months_Week3_Previous decimal(7,6),
MODIFY 0_Months_Week2_Previous decimal(7,6),
MODIFY 0_Months_Week1_Previous decimal(7,6),
MODIFY Fund VARCHAR(100),
MODIFY Milestone VARCHAR(100)
"""


In [7]:
pd.io.sql.execute(alter_table_query,conn)